In [ ]:
from IPython.display import clear_output
#from ipywidgets import interact, IntSlider

import os, os.path, shutil
import zipfile
import random
import json
import pickle
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import torch
import pypianoroll
import pretty_midi
from pypianoroll import Multitrack, Track
from tqdm import tqdm
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot

#from google.colab import drive

#drive.mount('/content/drive', force_remount=True)

In [ ]:
# Data
n_tracks = 4  # number of tracks
n_pitches = 83  # number of pitches
lowest_pitch = 24  # MIDI note number of the lowest pitch
n_samples_per_song = 8  # number of samples to extract from each song in the datset
n_measures = 4  # number of measures per sample
beat_resolution = 4  # temporal resolution of a beat (in timestep)
programs = [0, 0, 0, 0]  # program number for each track
is_drums = [False, False, False, False]  # drum indicator for each track
track_names = ['Soprano', 'Alto', 'Tenor', 'Bass']  # name of each track
tempo = 100

measure_resolution = 4 * beat_resolution
tempo_array = np.full((4 * 4 * measure_resolution, 1), tempo)

# Training
batch_size = 16
latent_dim = 128
n_steps = 1000

# Sampling
sample_interval = 10  # interval to run the sampler (in step)
n_samples = 4

#Directories
!mkdir -p midiDataset

In [ ]:
"""Midi dataset."""

from typing import Tuple
from torch import Tensor

import torch
from torch import nn
from torch.utils.data import Dataset

import numpy as np
from music21 import midi
from music21 import converter
from music21 import note, stream, duration, tempo


class MidiDataset(Dataset):
    """MidiDataset.
    Parameters
    ----------
    path: str
        Path to dataset.
    split: str, optional (default="train")
        Split of dataset.
    n_bars: int, optional (default=2)
        Number of bars.
    n_steps_per_bar: int, optional (default=16)
        Number of steps per bar.
    """

    def __init__(
        self,
        path: str,
        split: str = "train",
        n_bars: int = 8,
        n_steps_per_bar: int = 16,
    ) -> None:
        """Initialize."""
        self.n_bars = n_bars
        self.n_steps_per_bar = n_steps_per_bar
        dataset = np.load(path, allow_pickle=True, encoding="bytes")[split]
        self.data_binary, self.data_ints, self.data = self.__preprocess__(dataset)

    def __len__(self) -> int:
        """Return the number of samples in dataset."""
        return len(self.data_binary)

    def __getitem__(self, index: int) -> Tensor:
        """Return one samples from dataset.
        Parameters
        ----------
        index: int
            Index of sample.
        Returns
        -------
        Tensor:
            Sample.
        """
        return torch.from_numpy(self.data_binary[index]).float()

    def __preprocess__(self, data: np.ndarray) -> Tuple[np.ndarray]:
        """Preprocess data.
        Parameters
        ----------
        data: np.ndarray
            Data.
        Returns
        -------
        Tuple[np.ndarray]:
            Data binary, data ints, preprocessed data.
        """
        data_ints = []
        for x in data:
            skip = True
            skip_rows = 0
            while skip:
                if not np.any(np.isnan(x[skip_rows: skip_rows + 4])):
                    skip = False
                else:
                    skip_rows += 4
            #print(x.shape)
            if self.n_bars * self.n_steps_per_bar < x.shape[0]:
                data_ints.append(x[skip_rows: self.n_bars * self.n_steps_per_bar + skip_rows, :])
        
        
        data_ints = np.array(data_ints)

        


        #print(data_ints.shape)
        self.n_songs = data_ints.shape[0]
        self.n_tracks = data_ints.shape[2]
        data_ints = data_ints.reshape([self.n_songs, self.n_bars * self.n_steps_per_bar, self.n_tracks])
        #print(data_ints.shape)
        max_note = n_pitches
        mask = np.isnan(data_ints)
        data_ints[mask] = max_note + 1
        max_note = max_note + 1
        data_ints = data_ints.astype(int)
        #print(data_ints.shape)
        
        num_classes = max_note + 1
        data_binary = np.eye(num_classes)[data_ints]
        data_binary[data_binary == 0] = -1
        data_binary = np.delete(data_binary, max_note, -1)
        data_binary = data_binary.transpose([0, 1, 3, 2])
        #print(data_binary.shape)
        return data_binary, data_ints, data

In [ ]:
from pypianoroll.track import BinaryTrack
def save_pianoroll_as_midi(dataset,
                  programs=programs,
                  track_names=track_names,
                  is_drums=is_drums,
                  tempo=tempo,           # in bpm
                  beat_resolution=beat_resolution,  # number of time steps
                  destination_path="/content/midiDataset/"
                  ):
    data_ = []
    sopData = []

    for piece in dataset:

      pianoroll = piece > 0

      #print(pianoroll.shape)

    # Reshape batched pianoroll array to a single pianoroll array
      pianoroll_ = pianoroll.reshape((-1, pianoroll.shape[1], pianoroll.shape[2]))

      #print(pianoroll_.shape)

    # Create the tracks   
      tracks = []
      for idx in range(pianoroll_.shape[2]):
          tracks.append(pypianoroll.BinaryTrack(
            track_names[idx], programs[idx], is_drums[idx], pianoroll_[..., idx]))
          
      multitrack = pypianoroll.Multitrack(
          tracks=tracks, tempo=tempo_array, resolution=beat_resolution)
      
      data_.append(multitrack)

      melody = []
      for idx in range(1):
        melody.append(pypianoroll.BinaryTrack(
            track_names[idx], programs[idx], is_drums[idx], pianoroll_[..., idx]))
        
        sMultitrack = pypianoroll.Multitrack(
          tracks=melody, tempo=tempo_array, resolution=beat_resolution)
        
        sopData.append(sMultitrack)
        
      

    #print(tracks[0])
    #print(sData_)
      #print(multitrack)

    #melody = []
    #for track in (sopData):
      #melody.append(track)
      
      #sData_.append(sMultitrack)
    #print(sMultitrack)
    sopData[0].write('./midiDataset/test.mid')
    print('Midi saved to ', destination_path)
    print(tracks)
    #print(sopData)

    #print(data_)
    return data_, sopData

In [ ]:
from torch.utils import data
from numpy.core.fromnumeric import shape
d = np.load(r"C:\Users\lwgmi\Documents\GitHub\HonoursProject-reharmonisationGAN\Dataset\Jsb16thSeparated.npz", allow_pickle=True, encoding = 'latin1')

path = (r"C:\Users\lwgmi\Documents\GitHub\HonoursProject-reharmonisationGAN\Dataset\Jsb16thSeparated.npz")

dataset = MidiDataset(path=path).data_binary

train = d['train']
test = d['test']
valid = d['valid']

#print(train[3])

data, sData = save_pianoroll_as_midi(dataset)

mEvalData = data[100:200]

In [ ]:
print(data[3])

In [ ]:
class Evaluate():

    def __init__(self, piece: np.ndarray):

        self.piece = piece
        piece_unprocessed, pieceArr, Pianoroll = self.__PreProcess__(self.piece)

        self.piece_signature_vector = []

        self.piece_signature_vector.extend(
            [self.Number_of_notes(piece_unprocessed)[1], 
            self.Occupation_Rate(Pianoroll),
            self.Polyphonic_Rate(pieceArr, piece_unprocessed),
            self.Pitch_Range_Descriptors(piece_unprocessed)[0],
            self.Pitch_Range_Descriptors(piece_unprocessed)[1],
            self.Pitch_Range_Descriptors(piece_unprocessed)[2],
            self.Pitch_Range_Descriptors(piece_unprocessed)[3],
            self.Pitch_Interval_Range(piece_unprocessed)[0],
            self.Pitch_Interval_Range(piece_unprocessed)[1],
            self.Pitch_Interval_Range(piece_unprocessed)[2],
            self.Pitch_Interval_Range(piece_unprocessed)[3],
            self.Note_Duration(piece_unprocessed)[0],
            self.Note_Duration(piece_unprocessed)[1],
            self.Note_Duration(piece_unprocessed)[2],
            self.Note_Duration(piece_unprocessed)[3]
            ])
        


    def __PreProcess__(self, piece: np.ndarray):

        Pianoroll = piece.reshape(piece.shape[1], piece.shape[0], piece.shape[2])

        sop = []
        alt = []
        ten = []
        bass = []

        piece_unprocessed = []

        tPianoroll = (np.where(Pianoroll == True))

        sop = tPianoroll[2][0:128] 
        piece_unprocessed.append(sop)
        sop = [[i] for i in tPianoroll[2][0:128]]

        alt = tPianoroll[2][128:256] 
        piece_unprocessed.append(alt)
        alt = [[i] for i in tPianoroll[2][128:256]]

        ten = tPianoroll[2][256:384] 
        piece_unprocessed.append(ten)
        ten = [[i] for i in tPianoroll[2][256:384]]

        bass = tPianoroll[2][384:512] 
        piece_unprocessed.append(bass)
        bass = [[i] for i in tPianoroll[2][384:512]]

        pieceArr = [s + a + t + b for s, a, t, b in zip(sop, alt, ten, bass)]
        pieceArr = np.array(pieceArr)

        return piece_unprocessed, pieceArr, Pianoroll



    def Number_of_notes(self, piece_unprocessed: list):

        count = 0

        for track in piece_unprocessed:

            note = False
        

            for i, x in enumerate(track):

                pre = track[i-1]

                if i == 0:
                    pass

                elif pre == x and note == False: 

                    count = count + 1     
                    note = True

                elif pre == x and note == True:
                    pass

                elif pre != x and note == True:
                        note = False

                elif i == 128 and pre == x and note == False:
                    count = count + 1

                elif i == 128 and pre == x and note == True:
                    pass
                
        return count, count / 128



    def Occupation_Rate(self, Pianoroll: np.ndarray):

        occ_rate = 0

        for step in Pianoroll:
            if step.any() == True:
                occ_rate = occ_rate + 1
        else:
            pass

        return occ_rate / 128



    def Polyphonic_Rate(self, pieceArr: np.ndarray, piece_unprocessed):

        count = 0

        for i, timestep in enumerate(pieceArr[0::2]):

            pre = pieceArr[i - 1]

            if np.array_equal(timestep, pre) == False:
                count = count + 1

            elif np.array_equal(timestep, pre) == True:
                pass

        return count / self.Number_of_notes(piece_unprocessed)[0]



    def Pitch_Range_Descriptors(self, piece_unprocessed: list):#piece_unprocessed: list

        piece_unprocessed_whole = (*piece_unprocessed[0], *piece_unprocessed[1], *piece_unprocessed[2], *piece_unprocessed[3])
        
        max_note = max(piece_unprocessed_whole)
        min_note = min(piece_unprocessed_whole)
    
        mean = sum(piece_unprocessed_whole) / len(piece_unprocessed_whole)

        std_dev = np.std(piece_unprocessed_whole)

        return max_note / 127, min_note / 127, mean / 127, std_dev / 127



    def Pitch_Interval_Range(self, piece_unprocessed: list):

        interval = []

        for track in piece_unprocessed:

            note = False

            for i, x in enumerate(track):

                pre = track[i-1]

                if i == 0 or i ==(len(track) -1):
                    pass

                elif pre == x and note == False: 
                #count = count + 1
                    note = True

                elif pre == x and note == True:
                    pass

                elif pre != x and note == True:

                    interval.append(abs(pre - x))
                    note = False

        inv_max = max(interval)
        inv_min = min(interval)
    
        inv_mean = sum(interval) / len(interval)

        inv_std_dev = np.std(interval)

        return inv_max / 127, inv_min / 127, inv_mean / 127, inv_std_dev / 127


    def Note_Duration(self, piece_unprocessed: list):

        duration = []

        for track in piece_unprocessed:

            count = 0
                #print(duration)
                #print(len(track) - 1)

            for i, x in enumerate(track):

                pre = track[i-1]

                if i == 127:

                    count = count + 1
                    duration.append(count)

                elif pre != x and count != 0: #and note == True:

                    duration.append(count)
                    count = 1
            
                else:
                    if i == 0 or pre == x:
                        count = count + 1
 
        dur_max = max(duration)
        dur_min = min(duration)
    
        dur_mean = sum(duration) / len(duration)

        dur_std_dev = np.std(duration)
            #print(duration)

        return dur_max, dur_min, dur_mean, dur_std_dev

        

    def Mahalanobis_Distance(piece, mEvalData):

        x = np.array([Evaluate(piece).piece_signature_vector])

        mEvalVector = []

        for song in mEvalData:

            mPianoroll = song.stack()

            mEvalVector.append(Evaluate(mPianoroll).piece_signature_vector)

        mEvalVector = np.array(mEvalVector)

        m = np.mean(mEvalVector, axis=0)

        xMm = x - m

        mEvalVector = np.transpose(mEvalVector)

        covM = np.cov(mEvalVector, bias = False)

        invCovM = np.linalg.inv(covM)

        #np.set_printoptions(suppress= True)

        tem = np.dot(xMm, invCovM)
        tem2 = np.dot(tem, np.transpose(xMm))

        mD = np.reshape(np.sqrt(tem2), -1)

        return mD

In [ ]:
mahaData = []

for pianoroll in data[0:99]:

    pianoroll = pianoroll.stack()

    maha = Evaluate.Mahalanobis_Distance(pianoroll, mEvalData)

    mahaData.append(maha[0])



range = (0, 20)
bins = 20

print(mahaData)

plt.hist(mahaData, bins, range, color = 'red', histtype = 'bar', rwidth = 0.8)

plt.xlabel('Mahalanobis Distance')


plt.show()

mahaData = np.mean(mahaData)

print(mahaData)

